# <center><img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="600" />    
# <center>Decoding DDEC Reports Data
## <center>ENGR 580A2: Secure Vehicle and Industrial Networking
### <center> Department of Systems Engineering<br>Instructor: Dr. Jeremy Daily<br>Fall 2021

## Objective
This notebook is to accompany the analysis found in SAE 2015-01-1450 by Daily, et al.

The source file is a binary record found on one of the flash memory chips on a Detroit Diesel Series 60 Engine controller. The engine controller is in the DDEC5 family of controllers.

The data were extracted with a Xeltek Chip reader and named Chip2.bin. This file should be located in the same directory as this file.

In [1]:
import struct

In [2]:
#Let's load the chip binary into memory.
with open("Chip2.bin",'rb') as file_pointer:
    chip_memory = file_pointer.read()
print(chip_memory[:200])

In [3]:
#Locate the records in the binary
recordStart=867690 #for Chip2, as shown in the paper
recordLength = 5572

lastStopOffset = 1844
lastStopLength = 720

hardBrake1Offset = 896
hardBrake2Offset = 1370
hardBrakeLength = 450

In [4]:
# For last stop record
# Change these to match the Hard Brake to answer the quiz question.
start_index = recordStart + lastStopOffset
end_index = start_index + lastStopLength

last_stop_bytes = chip_memory[start_index:end_index]

# Print some raw hex
" ".join(["{:02X}".format(b) for b in last_stop_bytes])

'00 00 00 00 40 00 00 00 00 00 40 00 00 00 00 00 40 00 00 00 00 00 40 00 87 02 AD 00 40 00 0B 0C 7D 00 40 00 B3 0D 42 00 40 00 AA 0D 42 00 40 00 8F 0D 43 00 40 00 86 0D 43 00 40 00 82 0D 40 00 40 00 7A 0D 3F 00 40 00 71 0D 3F 00 40 00 61 0D 40 00 40 00 86 0D 3C 02 40 00 52 0D 40 04 00 00 46 0D 40 05 00 00 48 0D 3C 05 40 00 3F 0D 3D 06 40 00 41 0D 39 06 40 00 37 0D 3C 06 60 00 38 0D 3A 06 40 00 37 0D 3B 07 40 00 37 0D 3C 07 60 00 35 0D 3C 07 40 00 32 0D 3C 06 40 00 35 0D 3C 05 40 08 00 0C 8A 07 40 00 45 0D 4B 08 40 0F FC 0E 59 09 00 61 24 13 4F 0C 00 80 36 16 46 0E 00 77 85 16 3E 0E 00 70 95 14 25 0D 00 4B EC 0F 00 0C 00 00 9B 0D 64 0C 00 5B C3 10 56 0E 00 7B B6 12 4E 10 00 84 36 14 4B 11 00 74 4A 15 47 12 00 75 76 15 3D 12 00 6D F4 14 35 12 00 65 6B 14 38 12 00 65 03 14 38 11 00 64 CB 13 3B 11 00 66 E0 13 3F 11 00 69 F0 13 3C 11 00 68 EF 13 3B 11 00 67 2C 14 38 11 00 64 2B 14 38 11 00 64 17 14 38 11 00 64 EB 13 31 11 00 5E 35 14 32 11 00 5F 1A 14 2F 11 00 5C B9 13 31 11 00 5B 69 13 31 

Here's an image from the paper highlighting the same record. Use this to confirm you have the correct bytes. Notice the binary record starts as 00 00 00 00 40 and ends with a bunch of zeros. Visual inspections should demonstrate equivalence.

![Last Stop](LastStopOnChip2Capture.PNG)

Let's look for some patterns and match it to the DDEC Reports. Here's a page from the actual DDEC reports output:

![Missing file for Last Stop Table](DDECReportsLastStopTable.png)

There are 8 columns, other than time. There are no repeating elements in the record, so it's likely an array. There are 120 data entries in the table and there are 720 bytes in this record. The switch status are usually bit encoded, so let's look for patterns based on 6 bytes.

In [5]:
# Set the width of the data and print the bytes
width=6
for i in range(0,len(last_stop_bytes),width):
    print(" ".join(["{:02X}".format(b) for b in last_stop_bytes[i:i+width]]))

00 00 00 00 40 00
00 00 00 00 40 00
00 00 00 00 40 00
00 00 00 00 40 00
87 02 AD 00 40 00
0B 0C 7D 00 40 00
B3 0D 42 00 40 00
AA 0D 42 00 40 00
8F 0D 43 00 40 00
86 0D 43 00 40 00
82 0D 40 00 40 00
7A 0D 3F 00 40 00
71 0D 3F 00 40 00
61 0D 40 00 40 00
86 0D 3C 02 40 00
52 0D 40 04 00 00
46 0D 40 05 00 00
48 0D 3C 05 40 00
3F 0D 3D 06 40 00
41 0D 39 06 40 00
37 0D 3C 06 60 00
38 0D 3A 06 40 00
37 0D 3B 07 40 00
37 0D 3C 07 60 00
35 0D 3C 07 40 00
32 0D 3C 06 40 00
35 0D 3C 05 40 08
00 0C 8A 07 40 00
45 0D 4B 08 40 0F
FC 0E 59 09 00 61
24 13 4F 0C 00 80
36 16 46 0E 00 77
85 16 3E 0E 00 70
95 14 25 0D 00 4B
EC 0F 00 0C 00 00
9B 0D 64 0C 00 5B
C3 10 56 0E 00 7B
B6 12 4E 10 00 84
36 14 4B 11 00 74
4A 15 47 12 00 75
76 15 3D 12 00 6D
F4 14 35 12 00 65
6B 14 38 12 00 65
03 14 38 11 00 64
CB 13 3B 11 00 66
E0 13 3F 11 00 69
F0 13 3C 11 00 68
EF 13 3B 11 00 67
2C 14 38 11 00 64
2B 14 38 11 00 64
17 14 38 11 00 64
EB 13 31 11 00 5E
35 14 32 11 00 5F
1A 14 2F 11 00 5C
B9 13 31 11 00 5B
69 13 31 1

The pattern seems to have the first 2 bytes as Engine Speed in reverse byte order (Intel format).

The third byte seems to be engine load (reverse byte order).

The 4th byte is speed.

The 5th byte includes Clutch and Brake.

The 6th byte likely deals with the cruise and diagnostic code.

In [6]:
#Setup a data structure to process the binary record
dataValue={} # Initialize a dictionary
# Initialize lists for each record
dataValue['Road Speed']=[]
dataValue['Engine Speed']=[]
dataValue['Engine Load']=[]
dataValue['Throttle']=[]
dataValue['Brake Switch']=[]
dataValue['Clutch Switch']=[]
dataValue['Cruise Switch']=[]
dataValue['Diagnostic Code']=[]
dataValue['Time']=[]

In [9]:
# initialize the start time
t=-105 # 1 min and 45 seconds
width=6
rawBytes = last_stop_bytes
for i in range(0,len(rawBytes),width):
    # Increment the time
    t+=1.0
    dataValue['Time'].append(t)
    
    # Extract the data for Engine Speed
    rpmBytes=rawBytes[i:i+2]
    decodedRPM=struct.unpack('<H',rpmBytes)[0] * 0.25
    dataValue['Engine Speed'].append(decodedRPM)
    print(rpmBytes,decodedRPM)

    engineLoadBytes=rawBytes[i+2]
    decodedEngineLoad=engineLoadBytes * 0.5
    dataValue['Engine Load'].append(decodedEngineLoad)

    speedBytes=rawBytes[i+3]
    decodedSpeed=speedBytes * 0.5
    dataValue['Road Speed'].append(decodedSpeed)
    
    throttleBytes=rawBytes[i+5]
    decodedThrottleBytes=throttleBytes * 0.4
    dataValue['Throttle'].append(decodedThrottleBytes)
    
    bitField=rawBytes[i+4]
    
    # Use bit masking to determine switch states
    if bitField & 32 == 32:
       dataValue['Brake Switch'].append(1)
    else:
       dataValue['Brake Switch'].append(0)

    if bitField & 64 == 64:
      dataValue['Clutch Switch'].append(1)
    else:
      dataValue['Clutch Switch'].append(0)

    if bitField & 128 == 128: 
       dataValue['Cruise Switch'].append(1)
    else:
       dataValue['Cruise Switch'].append(0)

    if bitField & 1 == 1:
       dataValue['Diagnostic Code'].append(1)
    else:
       dataValue['Diagnostic Code'].append(0)


dataValue

b'\x00\x00' 0.0
b'\x00\x00' 0.0
b'\x00\x00' 0.0
b'\x00\x00' 0.0
b'\x87\x02' 161.75
b'\x0b\x0c' 770.75
b'\xb3\r' 876.75
b'\xaa\r' 874.5
b'\x8f\r' 867.75
b'\x86\r' 865.5
b'\x82\r' 864.5
b'z\r' 862.5
b'q\r' 860.25
b'a\r' 856.25
b'\x86\r' 865.5
b'R\r' 852.5
b'F\r' 849.5
b'H\r' 850.0
b'?\r' 847.75
b'A\r' 848.25
b'7\r' 845.75
b'8\r' 846.0
b'7\r' 845.75
b'7\r' 845.75
b'5\r' 845.25
b'2\r' 844.5
b'5\r' 845.25
b'\x00\x0c' 768.0
b'E\r' 849.25
b'\xfc\x0e' 959.0
b'$\x13' 1225.0
b'6\x16' 1421.5
b'\x85\x16' 1441.25
b'\x95\x14' 1317.25
b'\xec\x0f' 1019.0
b'\x9b\r' 870.75
b'\xc3\x10' 1072.75
b'\xb6\x12' 1197.5
b'6\x14' 1293.5
b'J\x15' 1362.5
b'v\x15' 1373.5
b'\xf4\x14' 1341.0
b'k\x14' 1306.75
b'\x03\x14' 1280.75
b'\xcb\x13' 1266.75
b'\xe0\x13' 1272.0
b'\xf0\x13' 1276.0
b'\xef\x13' 1275.75
b',\x14' 1291.0
b'+\x14' 1290.75
b'\x17\x14' 1285.75
b'\xeb\x13' 1274.75
b'5\x14' 1293.25
b'\x1a\x14' 1286.5
b'\xb9\x13' 1262.25
b'i\x13' 1242.25
b'\x87\x12' 1185.75
b'k\x12' 1178.75
b'\x10\x13' 1220.0
b'O\x13' 1235.7

{'Road Speed': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  2.0,
  2.5,
  2.5,
  3.0,
  3.0,
  3.0,
  3.0,
  3.5,
  3.5,
  3.5,
  3.0,
  2.5,
  3.5,
  4.0,
  4.5,
  6.0,
  7.0,
  7.0,
  6.5,
  6.0,
  6.0,
  7.0,
  8.0,
  8.5,
  9.0,
  9.0,
  9.0,
  9.0,
  8.5,
  8.5,
  8.5,
  8.5,
  8.5,
  8.5,
  8.5,
  8.5,
  8.5,
  8.5,
  8.5,
  8.5,
  8.5,
  8.0,
  8.0,
  8.0,
  8.5,
  8.5,
  9.0,
  9.0,
  9.5,
  9.5,
  9.5,
  10.0,
  10.0,
  10.5,
  10.0,
  10.0,
  10.0,
  9.5,
  9.5,
  9.5,
  9.0,
  8.5,
  8.0,
  7.0,
  6.5,
  5.0,
  4.5,
  4.0,
  4.0,
  4.5,
  5.5,
  5.0,
  4.5,
  4.5,
  4.0,
  3.5,
  3.0,
  2.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  2.0,
  2.5,
  2.5,
  3.0,
  3.0,


From this example, we can see how binary data is encoded, stored and decoded.

In [8]:
# Determine the maximum road speed
max(dataValue["Road Speed"])

10.5